In [1]:
#train rf to predict missing dereg_values

In [138]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [266]:
df = pd.read_csv('train_basic.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         16784 non-null  int64  
 1   listing_id         16784 non-null  int64  
 2   title              16784 non-null  object 
 3   make               16784 non-null  object 
 4   model              16784 non-null  object 
 5   description        16439 non-null  object 
 6   manufactured       16590 non-null  float64
 7   original_reg_date  318 non-null    object 
 8   reg_date           16583 non-null  object 
 9   type_of_vehicle    16784 non-null  object 
 10  category           16784 non-null  object 
 11  transmission       16784 non-null  object 
 12  curb_weight        16205 non-null  float64
 13  power              16784 non-null  float64
 14  fuel_type          3490 non-null   object 
 15  engine_cap         16784 non-null  float64
 16  no_of_owners       166

In [267]:
# calc = pd.read_csv('calculated_dereg.csv')
# df['dereg_value'] = calc['dereg_value']
# df.info()

In [243]:
# fill in missing omv
from collections import defaultdict
mal = []
dfm = df[df.omv.isnull()]
dfv = df[~df.omv.isnull()]
mapper = defaultdict(list)
for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 12 * 12
    ma = (model, age)
    mal.append((ma))

for idx, row in dfv.iterrows():
    model = row['model']
    age = row['age'] // 12 * 12
    ma = (model, age)
    if ma in mal:
        mapper[ma].append(row['omv'])

for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 12 * 12
    ma = (model, age)
    avg = np.average(mapper[ma])
    df.at[idx, 'omv'] = avg

print(df['omv'].isnull().sum())

33


In [244]:
print(np.max(df[df.omv.isnull()]['age']))
print(np.min(df[df.omv.isnull()]['age']))
print(np.mean(df[df.omv.isnull()]['age']))

876
4
539.030303030303


In [245]:
df_ = df.copy()
df_['omv'] = df['omv'].dropna()
# print(df_.loc[(df_.model == '530e'), 'omv'])

In [246]:
df.at[10005, 'omv'] = 58255

In [247]:
print(np.max(df[df.omv.isnull()]['age']))
print(np.min(df[df.omv.isnull()]['age']))
print(np.mean(df[df.omv.isnull()]['age']))

876
444
555.75


In [248]:
print(np.mean(df_[df_.age > 450]['omv']))

12575.818181818182


In [249]:
# fill in missing omv
from collections import defaultdict
mal = []
dfm = df[df.omv.isnull()]
dfv = df[~df.omv.isnull()]
mapper = defaultdict(list)
for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 120 * 120
    ma = (model, age)
    mal.append((ma))

for idx, row in dfv.iterrows():
    model = row['model']
    age = row['age'] // 120 * 120
    ma = (model, age)
    if ma in mal:
        mapper[ma].append(row['omv'])

for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 120 * 120
    ma = (model, age)
    avg = np.average(mapper[ma])
    df.at[idx, 'omv'] = avg

print(df['omv'].isnull().sum())

28


G:\softwares\envs\4277lab\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
G:\softwares\envs\4277lab\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [250]:
# fill in missing omv
from collections import defaultdict
mal = []
dfm = df[df.omv.isnull()]
dfv = df[~df.omv.isnull()]
mapper = defaultdict(list)
for idx, row in dfm.iterrows():
    model = row['make']
    age = row['age'] // 120 * 120
    ma = (model, age)
    mal.append(model)

for idx, row in dfv.iterrows():
    model = row['make']
    age = row['age'] // 120 * 120
    ma = (model, age)
    if model in mal:
        mapper[model].append(row['omv'])

for idx, row in dfm.iterrows():
    model = row['make']
    age = row['age'] // 120 * 120
    ma = (model, age)
    avg = np.average(mapper[model])
    df.at[idx, 'omv'] = avg

print(df['omv'].isnull().sum())

3


In [251]:
df.loc[df.omv.isnull(), 'omv'] = 12575
print(df['omv'].isnull().sum())

0


In [252]:
# coe: fill in missing value for coe according to: age and power
for idx, row in df.iterrows():
    omv = row['omv']
    if omv < 20000:
        arf = omv
    elif omv >= 20000 and omv <= 50000:
        arf = (omv - 20000) * 1.40 + 20000
    else:
        arf =  (omv - 50000) * 1.80 + 62000
        
    if pd.isnull(row['arf']):
        df.at[idx, 'arf'] = arf
# print(np.average(diff))
# df = df[~(df.arf.isnull() & df.dereg_value.isnull())] # remove values whose dereg_value cannot be calculated

In [253]:
df['remaining_coe'] = df['remaining_coe'].fillna(0)
# for date missing both coe and dereg_value, record it's month engine and power
coe_to_search = []
to_search = df[df.coe.isnull()]

for idx, row in to_search.iterrows():
    # category -> A / B
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["remaining_coe"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    coe_to_search.append((cat, month))

coe_to_search = list(set(coe_to_search))
print(len(coe_to_search))

14


In [254]:
t_to_coe = {}

for t in coe_to_search:
    t_to_coe[str(t)] = []

remains = df[~df.coe.isnull()]

for idx, row in remains.iterrows():
    if idx % 10000 == 0:
        print(idx)

    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["remaining_coe"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    
    t = (cat, month)
    
    coe = row["coe"]
    if t in coe_to_search:
        t_to_coe[str(t)].append(coe)
        
c = 0
for idx, row in df.iterrows():
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["remaining_coe"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    t = str((cat, month))

    if type(row["coe"]) == float and pd.isna(row["coe"]):
        if t in t_to_coe:
            l = t_to_coe[t]
            if len(l) > 0:
                avg = sum(l) / len(l)
                df.at[idx, 'coe'] = avg
                c += 1
print(c)

0
10000
891


In [255]:
print(df['coe'].isnull().sum())

0


In [256]:
# relevant attributes: omv, remaining coe, arf, power, age, omv, arf
useful = ['omv', 'arf', 'coe', 'remaining_coe', 'power', 'age']
train = df[~df.dereg_value.isnull()][useful]
y = df[~df.dereg_value.isnull()]['dereg_value']
to_predict = df[df.dereg_value.isnull()][useful]

In [257]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

In [262]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(random_state = 0, max_features = 4).fit(X_train,y_train)

In [263]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
mse=mean_squared_error(y_test, y_pred)
print(mse)
# 21582718 with dep
# 35585465

35585465.60370748


In [264]:
# new_dereg = model.predict(to_predict)
model=RandomForestRegressor(random_state = 0).fit(train,y)
new_dereg = model.predict(to_predict)

df.loc[df.dereg_value.isnull(), 'dereg_value'] = new_dereg
print(df['dereg_value'].isnull().sum())

0


In [265]:
df.to_csv('filled_dereg.csv')